In [ ]:
## 加载必备库文件

import numpy as np

import mxnet as mx
from mxnet import nd
from mxnet import gluon
from mxnet import autograd

from mxnet.gluon import nn
from mxnet.gluon import data as gdata
from mxnet.gluon.data.vision import transforms as gtransforms


from gluoncv import model_zoo

In [ ]:
## 损失函数
l1_loss = gluon.loss.L1Loss()
sigmoid_ce = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=False)
# 第一个参数是 pred ，第二个参数是 label
softmax_ce = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
## 性能度量函数
def metric_val(net, dataset_val, metric=None, ctx=mx.cpu(0)):
    
    if metric is None:
        metric = mx.metric.Accuracy()
        
    for _, batch in enumerate(dataset_val):
        # 将数据切片分别加载到不同的设备上
        data_val = gluon.utils.split_and_load(batch[0], ctx, batch_axis=0)
        label_val = gluon.utils.split_and_load(batch[1], ctx, batch_axis=0)
        
        yhat = [net(x) for x in data_val]
        
        # 第一个参数是 label ， 第二个参数是 output
        metric.update(label_val, yhat)    
  
    return metric.get()

In [ ]:
## 数据预处理
transform_train = gtransforms.Compose([
    # Randomly crop an area, and then resize it to be 32x32
    gtransforms.RandomResizedCrop(32),
    # Randomly flip the image horizontally
    gtransforms.RandomFlipLeftRight(),
    # Randomly jitter the brightness, contrast and saturation of the image
    gtransforms.RandomColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    # Randomly adding noise to the image
    gtransforms.RandomLighting(0.1),
    # Transpose the image from height*width*num_channels to num_channels*height*width
    # and map values from [0, 255] to [0,1]
    gtransforms.ToTensor(),
    # Normalize the image with mean and standard deviation calculated across all images
    gtransforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

transform_val = gtransforms.Compose([
    gtransforms.Resize(32),
    gtransforms.ToTensor(),
    gtransforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])


## 数据加载
# Dataset 类提供了两个转换函数： transform_first 和 transform ；
# transform_first 只变换 data ； transform 同时变换样本和标签（一个样本的所有数据）
loader_train = gdata.DataLoader(voc_train.transform(transforms_train), 
                              batch_size=batch_size, 
                              shuffle=True, 
                              last_batch='discard', 
                              num_workers=num_workers)

loader_val = gdata.DataLoader(voc_val.transform(transforms_val),
                            batch_size=batch_size,
                            shuffle=False, 
                            num_workers=num_workers)

for ib, batch in enumerate(loader_train):
    if ib > 0:
        break
    print('data:', batch[0][0].shape)
    print('label:', batch[6][0].shape)
    # 将数据切片分别加载到不同的设备上 ； 始终卡在这里，下一条打印信息没有出来
    data_train = gluon.utils.split_and_load(batch, ctx_list=ctx, batch_axis=0)
    
    print("aaa")

    with autograd.record():
        input_order = [0, 6, 1, 2, 3, 4, 5]
        obj_loss, center_loss, scale_loss, cls_loss = yolov3_model(*[data_train[o] for o in input_order])
        # sum up the losses
        # some standard gluon training steps:
        # autograd.backward(sum_loss)
        # trainer.step(batch_size)
        print("bbb")

In [ ]:
# 学习参数
# 调节学习速率衰减的倍数
lr_decay = 0.1
lr_decay_epochs_set = set([200, 400])

def update_learn_rate(trainer, epoch, lr_decay_epochs_set, lr_decay=0.1):
    if epoch in lr_decay_epochs_set:
        trainer.set_learning_rate(trainer.learning_rate * lr_decay)


from gluoncv.utils import TrainingHistory
train_history = TrainingHistory(['train', 'val'])


# 最优化
optimizer = mx.optimizer.Adam(learning_rate=0.0001,
                             beta1=0.9,
                             beta2=0.999,
                             epsilon=1e-08,
                             lazy_update=True)

trainer = gluon.Trainer(yolov3_model.collect_params(), optimizer)



In [ ]:
## 冻结原有层的权重
#resnet18_cifar10[0].collect_params().setattr('grad_req', 'null')
## 初始化自定义层的权重
#resnet18_cifar10[1].initialize(init=mx.init.Xavier(), ctx=ctx)

In [ ]:
## 网络训练
def train_net(net, data_train, data_val, trainer, epochs):
    for epoch in range(epochs):
        
        #metric.reset()
        train_loss = 0
        update_learn_rate(trainer, epoch, lr_decay_epochs, lr_decay)
        tic = time.time()
        
        # dataset 是可迭代对象
        for i, batch in enumerate(loader_train):
            # 将数据切片分别加载到不同的设备上
            data_train = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
            label_train = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
            
            # 自动求导，是因为这里的处理才导致 list 有 backward 函数？
            with autograd.record():
                output = [net(X) for X in data_train]
                loss = [loss_fn(yhat, y) for yhat, y in zip(output, label_train)]


            for l in loss:
                # loss 是一个 list ，为什么可以迭代？ 而且这里的 l 和 loss 是一样的？ 
                l.backward()
            
            # 这里更新的时候是怎样用到上面计算的 loss 的？
            # trainer 已经中指定了需要训练的参数
            trainer.step(batch_size)
                        
            for l in loss:
                train_loss += l.sum().asscalar() / batch_size
            
            # 每个 batch 更新一下训练的准确率
            metric.update(label_train, output)
            
        _, acc = metric.get()
        _, val_acc = metric_val(net, dataset_val, ctx=ctx)
        
        # 这里记录的是错误率
        train_history.update([1-acc, 1-val_acc])
        
        toc = time.time()
        
        print('[epoch %d] train_loss=%f, acc=%f, val_acc=%f, lr=%.9f, time: %fs' % 
              (epoch, train_loss, acc, val_acc, trainer.learning_rate, toc-tic))